In [1]:
#%env CUDA_VISIBLE_DEVICES=0

In [3]:
from collections import Counter
from glob import glob
from itertools import islice

import pandas as pd
import numpy as np
import scipy
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

from era_data import TabletPeriodDataset
from era_model import SimpleCNN

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# Hyperparameters

In [5]:
IMG_DIR = 'output/images_preprocessed'
VERSION_NAME = 'period_clf_bs16_lr1e-05_30epochs-vanillaCNN-94936_samples-preprocessed-April16-80-10-10_train-test-val'

#'period_clf_bs16_lr1e-05_30epochs-vanillaCNN-94936_samples-preprocessed-April08-70-10-20_train_test_val2'
#'period_clf_bs16_lr1e-05_10epochs-vanillaCNN-94936_samples-preprocessed-April06_1000test'
CKPT_FN = glob(f'lightning_logs/{VERSION_NAME}/checkpoints/*')[0]
CKPT_FN
BATCH_SIZE=16

In [6]:
num_classes = len(TabletPeriodDataset.PERIOD_INDICES)
num_classes

22

# Load data and model

In [7]:
test_ids = pd.read_csv(f'output/clf_ids/period-test-{VERSION_NAME}.csv', header=None)[0].astype(str)

In [8]:
ds_test = TabletPeriodDataset(IDS=test_ids, IMG_DIR=IMG_DIR)

Filtering 94936 IDS down to provided 9494...


In [9]:
def collate_fn(batch):
    data = torch.stack([torch.from_numpy(sample[1]).unsqueeze(0) for sample in batch])
    labels = torch.tensor([sample[2] for sample in batch])

    return data, labels

In [10]:
dl_test = DataLoader(ds_test, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False, num_workers=4)

In [11]:
%time model = SimpleCNN.load_from_checkpoint(CKPT_FN, num_classes=num_classes)

CPU times: user 2.32 s, sys: 4.97 s, total: 7.29 s
Wall time: 18.8 s


In [12]:
model.to(device);

In [13]:
model.eval();

# Calculate Predictions

In [14]:
def dl2data(dl, MAX_N=None, device='cuda'):
    logits = []
    y_true = []  # This will hold the period indices
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        gen = tqdm(islice(dl, MAX_N), total=(MAX_N if MAX_N is not None else len(dl)))
        for img, period_index in gen:
            try:
                y_true.append(period_index.cpu().numpy())  # Append period indices
                logits.append(model(img.to(device)).cpu().numpy())  # Process the image through the model
            except Exception as e:
                print(f"Error processing batch: {e}")
    y_true = np.hstack(y_true)
    logits = np.vstack(logits)

    return logits, y_true

In [15]:
logits, y_true = dl2data(dl_test)

100%|██████████| 582/582 [00:20<00:00, 28.71it/s]


In [16]:
y_true.shape, logits.shape

((9297,), (9297, 22))

In [17]:
y_pred = logits.argmax(axis=-1)
y_prob = scipy.special.softmax(logits, axis=-1)

In [18]:
y_pred.shape, y_prob.shape

((9297,), (9297, 22))

In [19]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.02      0.04        49
           1       0.81      0.93      0.87      2799
           2       0.95      0.98      0.96      2304
           3       0.58      0.79      0.67      1671
           4       0.63      0.33      0.43       602
           5       0.71      0.40      0.51       502
           6       0.85      0.49      0.62       304
           7       0.77      0.46      0.57       200
           8       0.77      0.27      0.40       205
           9       0.70      0.61      0.65       154
          10       0.62      0.42      0.50       149
          11       0.89      0.66      0.76       147
          12       0.79      0.48      0.59        23
          13       0.00      0.00      0.00         1
          14       0.73      0.62      0.67        64
          15       0.75      0.44      0.55        48
          16       0.50      0.06      0.10        18
          17       0.80    

/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kapond/.conda/envs/dani_torch/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

# Check Metrics

In [20]:
EARLY_BRONZE = {
    'Old Akkadian', 'Ur III',
    'ED IIIb', 'Uruk III',
    'Proto-Elamite', 'Lagash II',
    'Ebla', 'ED IIIa', 'ED I-II',
    'Uruk IV', 'Linear Elamite',
    'Harappan'
    
}
MID_LATE_BRONZE = {
    'Early Old Babylonian',
    'Old Babylonian', 'Old Assyrian',
    'Middle Babylonian', 'Middle Assyrian',
    'Middle Elamite', 'Middle Hittite'
}
IRON = {
    'Neo-Babylonian', 'Neo-Assyrian',
    'Achaemenid', 'Hellenistic',
    'Neo-Elamite'
}
ERA_MAP = {
    **{K: 'EB' for K in EARLY_BRONZE},
    **{K: 'MLB' for K in MID_LATE_BRONZE},
    **{K: 'I' for K in IRON},
}

In [21]:
def explain(period):
    return f'{period} ({ERA_MAP.get(period, "?")})'

In [22]:
idx2period = {v: k for k, v in TabletPeriodDataset.PERIOD_INDICES.items()}
idx2period[0] = 'other'

In [29]:
# let's just use classes with support >=10, everything else goes to 0: other
COMMON_LABELS = list({k for k, v in Counter(y_true).items() if v >= 40})
print(f'Common labels: ({len(COMMON_LABELS)})')
[(i, explain(idx2period[i])) for i in COMMON_LABELS]

Common labels: (14)


[(0, 'other (?)'),
 (1, 'Ur III (EB)'),
 (2, 'Neo-Assyrian (I)'),
 (3, 'Old Babylonian (MLB)'),
 (4, 'Middle Babylonian (MLB)'),
 (5, 'Neo-Babylonian (I)'),
 (6, 'Old Akkadian (EB)'),
 (7, 'Achaemenid (I)'),
 (8, 'Early Old Babylonian (MLB)'),
 (9, 'ED IIIb (EB)'),
 (10, 'Middle Assyrian (MLB)'),
 (11, 'Old Assyrian (MLB)'),
 (14, 'Lagash II (EB)'),
 (15, 'Ebla (EB)')]

In [30]:
y_true_c = y_true.copy()
y_true_c[~np.isin(y_true, COMMON_LABELS)] = 0
print((~np.isin(y_true, COMMON_LABELS)).mean(), 'changed to "other"')

0.010648596321393998 changed to "other"


In [31]:
y_pred_c = y_pred.copy()
y_pred_c[~np.isin(y_pred, COMMON_LABELS)] = 0
print((~np.isin(y_pred, COMMON_LABELS)).mean(), 'changed to "other"')

0.00484027105517909 changed to "other"


In [32]:
indices_c = list(set(y_true_c) | set(y_pred_c))
print(len(indices_c))
print(indices_c)
PERIOD_LABELS_C = [explain(idx2period[i]) for i in indices_c]
print(PERIOD_LABELS_C)

14
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15]
['other (?)', 'Ur III (EB)', 'Neo-Assyrian (I)', 'Old Babylonian (MLB)', 'Middle Babylonian (MLB)', 'Neo-Babylonian (I)', 'Old Akkadian (EB)', 'Achaemenid (I)', 'Early Old Babylonian (MLB)', 'ED IIIb (EB)', 'Middle Assyrian (MLB)', 'Old Assyrian (MLB)', 'Lagash II (EB)', 'Ebla (EB)']


In [33]:
print(classification_report(y_true_c, y_pred_c, target_names=PERIOD_LABELS_C))

                            precision    recall  f1-score   support

                 other (?)       0.64      0.20      0.31       148
               Ur III (EB)       0.81      0.93      0.87      2799
          Neo-Assyrian (I)       0.95      0.98      0.96      2304
      Old Babylonian (MLB)       0.58      0.79      0.67      1671
   Middle Babylonian (MLB)       0.63      0.33      0.43       602
        Neo-Babylonian (I)       0.71      0.40      0.51       502
         Old Akkadian (EB)       0.85      0.49      0.62       304
            Achaemenid (I)       0.77      0.46      0.57       200
Early Old Babylonian (MLB)       0.77      0.27      0.40       205
              ED IIIb (EB)       0.70      0.61      0.65       154
     Middle Assyrian (MLB)       0.62      0.42      0.50       149
        Old Assyrian (MLB)       0.89      0.66      0.76       147
            Lagash II (EB)       0.73      0.62      0.67        64
                 Ebla (EB)       0.75      0.44

In [34]:
Y = label_binarize(y_true, classes=np.unique(y_true))
n_classes = Y.shape[1]

auc_scores = []
for i in range(n_classes):
    auc = roc_auc_score(Y[:, i], y_prob[:, i])
    auc_scores.append(auc)

macro_ovr_auc = np.mean(auc_scores)
print(f"Macro-OvR-AUC: {macro_ovr_auc}")

Macro-OvR-AUC: 0.9335689829895032
